# LTV Heatmap
Have to learn how to make an LTV Heatmap using Cohort analysis?

In [4]:
# Libraries
import pandas as pd
import datetime as dt

In [2]:
# load data
df = pd.read_csv('payments.csv')

### Preliminary Exploration

In [3]:
df.head()

,account_id,payment_id,payment_usd,payment_date
0,hbQ5GR1L54QtQAB,QhQfR06N8dQAO,50.00,2021-01-28
1,vsQ5GR1KF4QtQAH,Q7QfR06N8dQAO,968.07,2021-01-28
2,5jQ5GR1Uz4QtQAB,Q6QfR06N8dQAO,763.67,2021-01-28
3,hnQ5GR1O34QtQAB,Q2QfR06N8dQAO,2378.57,2021-01-28
4,ztQ5GR1Ha4QtQAF,Q0QfR06N8dQAO,2855.84,2021-01-28


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6890 entries, 0 to 6889
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   account_id    6890 non-null   object 
 1   payment_id    6890 non-null   object 
 2   payment_usd   6890 non-null   float64
 3   payment_date  6890 non-null   object 
dtypes: float64(1), object(3)
memory usage: 215.4+ KB


In [ ]:
user_counts = pd.value_counts(df['account_id'])
user_counts.head()

xqQ5GR06N4QtQAC    28
GzQ5GR1Fn4QtQAJ    27
nUQ5GR1EK4QtQAT    25
RcQ5GR0p44QtQAI    24
cwQ5GR15j4QtQAE    24
Name: account_id, dtype: int64

In [21]:
payment_id_counts = pd.value_counts(df['payment_id']).to_frame().rename(columns={'payment_id':'count'})
payment_id_counts

,count
CBQfR02t8dQA2,2
BtQfR02t8dQAM,2
AmQfR02t8dQAM,2
i4QfR05g8dQAA,2
i5QfR05g8dQAA,2
...,...
hwQfR0638dQAE,1
hxQfR0638dQAE,1
hyQfR0638dQAE,1
hzQfR0638dQAE,1


Weird....some of the payment IDs are duplicates, but theyre associated with different accounts?

### Cleaning / Manipulating 
Things to do:
- Convert `payment_date` column to datetime
- Add YearMonth Column
- 

In [19]:
# look for missing values
df.isna().sum()

account_id      0
payment_id      0
payment_usd     0
payment_date    0
dtype: int64

We saw before that there are some payment duplicates. Let's get a list of all the different duplicates


In [ ]:
# View DF of duplicated payment id
payment_id_counts.loc[payment_id_counts['count'] > 1]

,count
CBQfR02t8dQA2,2
BtQfR02t8dQAM,2
AmQfR02t8dQAM,2
i4QfR05g8dQAA,2
i5QfR05g8dQAA,2
...,...
56QfR02t8dQAE,2
55QfR02t8dQAE,2
G6QfR05g8dQAI,2
AkQfR02t8dQAM,2


In [ ]:
#list of all the duplicated payment ids
payment_dup_id = payment_id_counts.loc[payment_id_counts['count'] > 1].index.to_list()

In [11]:
# Get DataFrame with only these payment ids
df.loc[df.duplicated(subset='payment_id', keep=False)].sort_values(by='payment_id')

,account_id,payment_id,payment_usd,payment_date
3821,O1Q5GR1SY4QtQAL,3iQfR02t8dQAE,125.00,2020-09-12
3378,10Q5GR0V04QtQAB,3iQfR02t8dQAE,14829.55,2020-09-30
3762,XrQ5GR1SZ4QtQAP,3sQfR02t8dQAE,250.00,2020-09-15
3842,aoQ5GR18t4QtQAA,3sQfR02t8dQAE,50.00,2020-09-11
3761,JQQ5GR1Rk4QtQAZ,3tQfR02t8dQAE,250.00,2020-09-15
...,...,...,...,...
5154,QmQ5GR1Ie4QtQAN,vtQfR05h8dQAE,250.00,2020-07-04
3896,YWQ5GR1L64QtQA3,wHQfR02t8dQAU,500.10,2020-09-08
3111,IfQ5GR0p64QtQAM,wHQfR02t8dQAU,2728.06,2020-10-09
1321,IfQ5GR1SC4QtQAL,yWQfR06N8dQAS,75.00,2020-12-10


In [15]:
print(f"We had {279} duplicates, each appearing {2} times for a total of 279*2={279*2}. This checks out")

We had 279 duplicates, each appearing 2 times for a total of 279*2=558. This checks out


We can see above that even though the `payment_id` are duplicated (twice for each), they appear to be associated with different values, dates, and accounts. Therefore it is justified to be keeping this data in our dataset, and treat the duplication as an artifact of the data entry.